In [1]:
from google.colab import drive

drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/images'
import os
print(os.listdir(folder_path))

Mounted at /content/drive
['006_HC.png', '003_HC.png', '010_2HC.png', '002_HC.png', '011_HC.png', '009_HC.png', '010_HC.png', '005_HC.png', '007_HC.png', '004_HC.png', '008_HC.png', '000_HC.png', '001_HC.png', '013_HC.png', '014_3HC.png', '019_HC.png', '014_2HC.png', '014_HC.png', '012_HC.png', '017_2HC.png', '016_HC.png', '018_HC.png', '019_2HC.png', '015_HC.png', '017_HC.png', '025_HC.png', '026_2HC.png', '020_HC.png', '023_HC.png', '026_HC.png', '021_HC.png', '023_2HC.png', '027_HC.png', '022_2HC.png', '022_HC.png', '028_HC.png', '024_HC.png', '030_HC.png', '033_HC.png', '037_HC.png', '036_HC.png', '035_HC.png', '032_HC.png', '032_2HC.png', '029_HC.png', '034_HC.png', '038_HC.png', '031_HC.png', '039_HC.png', '033_2HC.png', '043_HC.png', '042_HC.png', '040_HC.png', '047_HC.png', '046_HC.png', '045_HC.png', '041_HC.png', '044_HC.png', '049_HC.png', '048_HC.png', '056_HC.png', '054_HC.png', '053_HC.png', '051_HC.png', '050_2HC.png', '052_HC.png', '060_HC.png', '050_HC.png', '055_HC.pn

In [5]:
import os
from PIL import Image
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import random_split

# Define the dataset class
class LandmarkDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, target_size=(256, 256)):
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_size = target_size

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.landmarks_frame.iloc[idx, 0])
        image = Image.open(img_name).convert('L')
        image = image.resize(self.target_size)

        landmarks = self.landmarks_frame.iloc[idx, 1:].values.astype('float').reshape(-1, 2)

        if self.transform:
            image = self.transform(image)

        return {'image': image, 'landmarks': landmarks}

# Defining the ResNet-based landmark detection model
class LandmarkDetectionModel(nn.Module):
    def __init__(self, num_classes=8):
        super(LandmarkDetectionModel, self).__init__()
        resnet = models.resnet18(pretrained=True)
        # Remove the last layer (the fully connected layer)
        self.resnet_features = nn.Sequential(*list(resnet.children())[:-1])
        # Add custom fully connected layers for landmark detection
        self.fc1 = nn.Linear(resnet.fc.in_features, 100)
        self.fc2 = nn.Linear(100, num_classes)

    def forward(self, x):
        features = self.resnet_features(x)
        features = features.view(features.size(0), -1)  # Flatten the feature map
        x = torch.relu(self.fc1(features))
        x = self.fc2(x)
        return x

# Paths to CSV file and image directory
csv_file = '/content/drive/MyDrive/role_challenge_dataset_ground_truth.csv'
root_dir = '/content/drive/MyDrive/images'

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


landmark_dataset = LandmarkDataset(csv_file=csv_file, root_dir=root_dir, transform=transform)


model = LandmarkDetectionModel()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Split dataset into train, validation, and test sets
train_size = int(0.6 * len(landmark_dataset))
val_size = int(0.2 * len(landmark_dataset))
test_size = len(landmark_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(landmark_dataset, [train_size, val_size, test_size])

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, batch in enumerate(train_loader):
        images, landmarks = batch['image'], batch['landmarks']
        optimizer.zero_grad()
        outputs = model(images)
        landmarks = landmarks.view(-1, 8)
        loss = criterion(outputs, landmarks.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {epoch_loss:.4f}')

Epoch [1/10], Batch [1/3], Loss: 137061.6562
Epoch [1/10], Batch [2/3], Loss: 144366.7969
Epoch [1/10], Batch [3/3], Loss: 140159.8906
Epoch [1/10], Training Loss: 140540.3464
Epoch [2/10], Batch [1/3], Loss: 138339.4688
Epoch [2/10], Batch [2/3], Loss: 139225.9219
Epoch [2/10], Batch [3/3], Loss: 140606.2188
Epoch [2/10], Training Loss: 139354.6852
Epoch [3/10], Batch [1/3], Loss: 139337.5938
Epoch [3/10], Batch [2/3], Loss: 137496.8438
Epoch [3/10], Batch [3/3], Loss: 136723.6406
Epoch [3/10], Training Loss: 137885.9892
Epoch [4/10], Batch [1/3], Loss: 138593.2500
Epoch [4/10], Batch [2/3], Loss: 136944.8750
Epoch [4/10], Batch [3/3], Loss: 132001.4219
Epoch [4/10], Training Loss: 135959.9098
Epoch [5/10], Batch [1/3], Loss: 135786.2812
Epoch [5/10], Batch [2/3], Loss: 133256.0156
Epoch [5/10], Batch [3/3], Loss: 131215.4219
Epoch [5/10], Training Loss: 133484.2315
Epoch [6/10], Batch [1/3], Loss: 131599.5469
Epoch [6/10], Batch [2/3], Loss: 128397.5156
Epoch [6/10], Batch [3/3], Los

In [8]:
# Testing loop
model.eval()
test_loss = 0.0
with torch.no_grad():
    for batch_idx, batch in enumerate(test_loader):
        images, landmarks = batch['image'], batch['landmarks']
        outputs = model(images)
        landmarks = landmarks.view(-1, 8)
        loss = criterion(outputs, landmarks.float())
        test_loss += loss.item() * images.size(0)
        print(f'Test Batch [{batch_idx + 1}/{len(test_loader)}], Loss: {loss.item():.4f}')
test_loss /= len(test_dataset)
print(f'Test Loss: {test_loss:.4f}')

Test Batch [1/1], Loss: 104498.4609
Test Loss: 104498.4609


In [9]:
# Validation loop
val_loss = 0.0
with torch.no_grad():
    for batch_idx, batch in enumerate(val_loader):
        images, landmarks = batch['image'], batch['landmarks']
        outputs = model(images)
        landmarks = landmarks.view(-1, 8)
        loss = criterion(outputs, landmarks)
        val_loss += loss.item() * images.size(0)
        print(f'Validation Batch [{batch_idx + 1}/{len(val_loader)}], Loss: {loss.item():.4f}')
val_loss /= len(val_dataset)
print(f'Validation Loss: {val_loss:.4f}')

Validation Batch [1/1], Loss: 106469.9117
Validation Loss: 106469.9117


In [10]:
# Inference with the trained ResNet model
single_image_path = '/content/drive/MyDrive/images/000_HC.png'
single_image = Image.open(single_image_path).convert('L')
single_image = single_image.resize((256, 256))
single_image = transform(single_image).unsqueeze(0)


with torch.no_grad():
    model.eval()
    predicted_landmarks = model(single_image)


print("Predicted Landmarks:")
print(predicted_landmarks)


Predicted Landmarks:
tensor([[69.2831, 30.3530, 58.8354, 70.7256, 58.3452, 55.1645, 55.3002, 31.8154]])
